### Script de definição dos limites de DEC e FEC
* Autor: Leonardo da Silveira
* Versão 2.0

##### Conexão com o Google Drive

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    caminho = '/content/drive/MyDrive/Código e simulações - Python/'
except:
    caminho = ''

##### Importação das Bibliotecas

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats

##### Definição das variáveis iniciais

In [3]:
qtdConjSemDesej = 100  # Nº desejável de conj semelhantes
qtdConjSemMin = 50     # Nº mínimo de conj semelhantes
hetMax = 20            # Heterogeneidade máxima [%];

p1 = 0.2     # Percentil base SIN/aéreo
p2 = 0.5     # Percentil base isolado/subt

## Conjuntos a serem processados
distInteresse = '' #Distribuidora de interesse

## Atributos utilizados para DEC e FEC
atrDec = ['PC_VRAM','PLUV','PC_ERMT_3F','PC_NUC_AD','CM_NUC_RES']
atrFec = ['PC_VRAM','PLUV','PC_ERMT_3F','PC_NUC_AD','CM_NUC_RES']

## Atributos cuja complexidade aumenta conforme se eleva o valor do atributo
atrAniPos = ['PC_VRAM','PLUV']

## Colunas contendo os valores de DEC e FEC apurados
decApurado = ['DEC_Ano1','DEC_Ano2','DEC_Ano3']
fecApurado = ['FEC_Ano1','FEC_Ano2','FEC_Ano3']

atributos = list(set(atrDec + atrFec)) # Lista de atributos
qtdAtrDec = len(atrDec) # Quantidade de atributos para DEC
qtdAtrFec = len(atrFec) # Quantidade de atributos para FEC

## Sufixo do nome dos arquivos de saída
sufixo = '_2021' #Ex.: '_caso1'

##### Importação dos dados de entrada

In [4]:
data = pd.read_excel(caminho + 'Atributos.xlsx')
data.set_index('Cod_Conj', inplace=True)
data.drop(columns=['DistribuidoraConjunto','Regiao','CcMed_Ano1','CcMed_Ano2','CcMed_Ano3'], inplace=True)

##### Remover conjuntos da clusterização

In [5]:
if data[atributos].isna().values.any():
    data.dropna(subset = atributos, inplace=True)
    print('Foram removidos conjuntos com atributos faltantes.')

##### DEC e FEC médios

In [6]:
data['DEC'] = data[decApurado].replace(0, np.nan).mean(axis=1, skipna = True)
data['FEC'] = data[fecApurado].replace(0, np.nan).mean(axis=1, skipna = True)
data['DEC_FEC'] = data['DEC']/data['FEC']

##### Atributo Normalizado Individual

In [7]:
medAtr = data[atributos].mean(axis=0, skipna = True)
desAtr = data[atributos].std(axis=0, skipna = True)
maxAtr = data[atributos].where(data[atributos] < medAtr + 3 * desAtr, np.nan).max(axis=0, skipna = True)
minAtr = data[atributos].min(axis=0, skipna = True)
ani = 100-100*(data[atributos]-minAtr)/(maxAtr-minAtr)
ani[atrAniPos] = 100-ani[atrAniPos]

##### Conjuntos aéreos/subterrâneos

In [8]:
data['Subt'] = data['Padrao'].apply(lambda x: True if x==2 or x==3 else False)

##### Normalização z-score

In [9]:
atribNorm = stats.zscore(data[atributos], axis=0, ddof=1)

##### Conjuntos a serem processados

In [10]:
dataInteresse = data[data['Distribuidora']==distInteresse] if distInteresse else data

##### Definição dos conjuntos semelhantes DEC

In [11]:
resultadoDec = pd.DataFrame()
conjSemDec = pd.DataFrame()

for index, row in dataInteresse.iterrows():
  
  a = (atribNorm[atrDec]-atribNorm[atrDec].loc[index]).pow(2).sum(axis=1).pow(1./2)*100/(3 * np.sqrt(qtdAtrDec))
  a = a.sort_values().to_frame(name='HeterogeneidadeDEC')
  a['CodConjInteresse']=index
  a['ConjInteresse']=row['Conjunto']
  a['CodConjSemeDEC']=a.index

  a = a.join(data[['Conjunto','Distribuidora','Padrao','Localizacao','Subt','DEC']])
  a.drop(a[a['Subt'] != row['Subt']].index, inplace=True)
  a = a[:qtdConjSemDesej]
  while (a['HeterogeneidadeDEC'].iloc[-1] > hetMax) and len(a.index)>qtdConjSemMin:
    a = a[:-1]
  conjSemDec = pd.concat([conjSemDec,a[['CodConjInteresse','ConjInteresse','HeterogeneidadeDEC','CodConjSemeDEC']]], 
                         ignore_index=True, sort=False)

  hmg = a['HeterogeneidadeDEC'].iloc[-1]
  aniMedio = ani[atrDec].loc[a.index].mean(axis=0, skipna = True)
  scoreAni = (ani[atrDec].loc[index] - aniMedio).sum()/qtdAtrDec

  percentil = p2 if row['Subt'] or row['Localizacao'] == 2 else p1

  if hmg > hetMax:
    if       scoreAni < -3 : percentil-=0.1
    if -3 <= scoreAni <  3 : percentil+=0
    if  3 <= scoreAni <  6 : percentil+=0.1
    if  6 <= scoreAni <  9 : percentil+=0.2
    if  9 <= scoreAni      : percentil+=0.3

  qtdConjSem = len(a.index)
  idxConjRef = np.floor((qtdConjSem - 1) * percentil + 1).astype(int)
  conjRef = a.sort_values(by=['DEC']).iloc[[idxConjRef-1]]
  limObj = np.ceil(conjRef['DEC'].iloc[0])

  d = {
      'ConjuntosSemelhantesDEC': qtdConjSem,
      'HeterogeneidadeDEC': hmg,
      'Score_ANI_DEC': scoreAni,
      # 'Percentil': 'P' + str(int(100*p)),
      'Conjunto_ReferênciaDEC_ANI': conjRef['Conjunto'].iloc[0],
      'Cod_Conj_Ref_DEC': conjRef.index,
      # 'DEC_Conj_Ref_DEC': conjRef['DEC'].iloc[0],
      'DEC_Lim_Obj': limObj,
      }
  resultadoDec = pd.concat([resultadoDec, pd.DataFrame(data=d, index=[index])])
resultadoDec

,ConjuntosSemelhantesDEC,HeterogeneidadeDEC,Score_ANI_DEC,Conjunto_ReferênciaDEC_ANI,Cod_Conj_Ref_DEC,DEC_Lim_Obj
12584,50,30.166801,2.704587,Jordão,12600,29.0
13002,96,19.971862,-0.868847,RENASCENCA,15764,5.0
15799,100,18.475053,-0.273988,BONJI,16182,5.0
13004,100,19.986254,-0.513328,BONJI,16182,5.0
15800,50,20.751971,-0.422448,TAMBAÚ,13741,5.0
...,...,...,...,...,...,...
15602,100,13.976383,0.828449,NOSSA SENHORA DAS DORES,14545,11.0
15601,100,19.111534,1.922598,Minas Novas 1,15272,12.0
12737,100,19.532867,1.998197,MORRO DO CHAPEU,14357,11.0
12743,100,17.312414,3.324149,Cláudio 1,15157,11.0


##### Definição dos conjuntos semelhantes FEC

In [12]:
resultadoFec = pd.DataFrame()
conjSemFec = pd.DataFrame()

for index, row in dataInteresse.iterrows():
  
  a = (atribNorm[atrFec]-atribNorm[atrFec].loc[index]).pow(2).sum(axis=1).pow(1./2)*100/(3 * np.sqrt(qtdAtrFec))
  a = a.sort_values().to_frame(name='HeterogeneidadeFEC')
  a['CodConjInteresse']=index
  a['ConjInteresse']=row['Conjunto']
  a['CodConjSemeFEC']=a.index

  a = a.join(data[['Conjunto','Distribuidora','Padrao','Localizacao','Subt','FEC']])
  a.drop(a[a['Subt'] != row['Subt']].index, inplace=True)
  a = a[:qtdConjSemDesej]
  while (a['HeterogeneidadeFEC'].iloc[-1] > hetMax) and len(a.index)>qtdConjSemMin:
    a = a[:-1]
  conjSemFec = pd.concat([conjSemFec,a[['CodConjInteresse','ConjInteresse','HeterogeneidadeFEC','CodConjSemeFEC']]], 
                         ignore_index=True, sort=False)

  hmg = a['HeterogeneidadeFEC'].iloc[-1]
  aniMedio = ani[atrFec].loc[a.index].mean(axis=0, skipna = True)
  scoreAni = (ani[atrFec].loc[index] - aniMedio).sum()/qtdAtrFec

  percentil = p2 if row['Subt'] or row['Localizacao'] == 2 else p1

  if hmg > hetMax:
    if       scoreAni < -3 : percentil-=0.1
    if -3 <= scoreAni <  3 : percentil+=0
    if  3 <= scoreAni <  6 : percentil+=0.1
    if  6 <= scoreAni <  9 : percentil+=0.2
    if  9 <= scoreAni      : percentil+=0.3

  qtdConjSem = len(a.index)
  idxConjRef = np.floor((qtdConjSem - 1) * percentil + 1).astype(int)
  conjRef = a.sort_values(by=['FEC']).iloc[[idxConjRef-1]]
  limObj = np.ceil(conjRef['FEC'].iloc[0])

  d = {
      'ConjuntosSemelhantesFEC': qtdConjSem,
      'HeterogeneidadeFEC': hmg,
      'Score_ANI_FEC': scoreAni,
      # 'Percentil': 'P' + str(int(100*p)),
      'Conjunto_ReferênciaFEC_ANI': conjRef['Conjunto'].iloc[0],
      'Cod_Conj_Ref_FEC': conjRef.index,
      # 'FEC_Conj_Ref_FEC': conjRef['DEC'].iloc[0],
      'FEC_Lim_Obj': limObj,
      }
  resultadoFec = pd.concat([resultadoFec, pd.DataFrame(data=d, index=[index])])
resultadoFec

,ConjuntosSemelhantesFEC,HeterogeneidadeFEC,Score_ANI_FEC,Conjunto_ReferênciaFEC_ANI,Cod_Conj_Ref_FEC,FEC_Lim_Obj
12584,50,30.166801,2.704587,COMODORO,14795,14.0
13002,96,19.971862,-0.868847,VILA NOVA,13557,3.0
15799,100,18.475053,-0.273988,CRUZ DO PEIXE,13696,3.0
13004,100,19.986254,-0.513328,CRUZ DO PEIXE,13696,3.0
15800,50,20.751971,-0.422448,CANDEAL,14272,3.0
...,...,...,...,...,...,...
15602,100,13.976383,0.828449,Santo Antonio de Padua,13080,6.0
15601,100,19.111534,1.922598,Dores do Indaiá,15347,5.0
12737,100,19.532867,1.998197,Patos de Minas 2,16318,5.0
12743,100,17.312414,3.324149,CATENDE,16176,5.0


##### Cálculo da trajetória de limites

In [13]:
trajLimites = pd.DataFrame()

for index, row in dataInteresse.iterrows():
  decLimObj = resultadoDec['DEC_Lim_Obj'].loc[index]
  fecLimObj = resultadoFec['FEC_Lim_Obj'].loc[index]

  if row['DECL_V0'] > decLimObj:
    trajConj = {
        'DEC_V1': np.around(row['DECL_V0'] - 1 * (row['DECL_V0'] - decLimObj) / 8),
        'DEC_V2': np.around(row['DECL_V0'] - 2 * (row['DECL_V0'] - decLimObj) / 8),
        'DEC_V3': np.around(row['DECL_V0'] - 3 * (row['DECL_V0'] - decLimObj) / 8),
        'DEC_V4': np.around(row['DECL_V0'] - 4 * (row['DECL_V0'] - decLimObj) / 8),
        'DEC_V5': np.around(row['DECL_V0'] - 5 * (row['DECL_V0'] - decLimObj) / 8),
    }
  else:
    trajConj = {
        'DEC_V1': row['DECL_V0'],
        'DEC_V2': row['DECL_V0'],
        'DEC_V3': row['DECL_V0'],
        'DEC_V4': row['DECL_V0'],
        'DEC_V5': row['DECL_V0'],
    }
  if row['FECL_V0'] > fecLimObj:
    trajConj.update({
        'FEC_V1': np.around(row['FECL_V0'] - 1 * (row['FECL_V0'] - fecLimObj) / 8),
        'FEC_V2': np.around(row['FECL_V0'] - 2 * (row['FECL_V0'] - fecLimObj) / 8),
        'FEC_V3': np.around(row['FECL_V0'] - 3 * (row['FECL_V0'] - fecLimObj) / 8),
        'FEC_V4': np.around(row['FECL_V0'] - 4 * (row['FECL_V0'] - fecLimObj) / 8),
        'FEC_V5': np.around(row['FECL_V0'] - 5 * (row['FECL_V0'] - fecLimObj) / 8),
    })
  else:
    trajConj.update({
        'FEC_V1': row['FECL_V0'],
        'FEC_V2': row['FECL_V0'],
        'FEC_V3': row['FECL_V0'],
        'FEC_V4': row['FECL_V0'],
        'FEC_V5': row['FECL_V0'],
    })

  trajLimites = pd.concat([trajLimites, pd.DataFrame(data=trajConj, index=[index])])

trajLimites.index.name='Cod_Conj'
trajLimites

,DEC_V1,DEC_V2,DEC_V3,DEC_V4,DEC_V5,FEC_V1,FEC_V2,FEC_V3,FEC_V4,FEC_V5
Cod_Conj,,,,,,,,,,
12584,69.0,64.0,58.0,52.0,46.0,56.0,50.0,44.0,38.0,32.0
13002,15.0,13.0,12.0,10.0,9.0,11.0,10.0,9.0,8.0,6.0
15799,21.0,18.0,16.0,14.0,12.0,14.0,12.0,10.0,9.0,8.0
13004,27.0,24.0,21.0,18.0,14.0,17.0,15.0,13.0,11.0,9.0
15800,30.0,26.0,22.0,19.0,16.0,17.0,15.0,13.0,11.0,9.0
...,...,...,...,...,...,...,...,...,...,...
15602,14.0,13.0,13.0,12.0,12.0,9.0,8.0,8.0,8.0,7.0
15601,16.0,15.0,14.0,14.0,14.0,9.0,9.0,8.0,8.0,7.0
12737,15.0,15.0,14.0,14.0,13.0,8.0,8.0,8.0,7.0,6.0


##### Resultado

In [14]:
resultado = dataInteresse[['Conjunto','Cod_Dist','Distribuidora']]
resultado = resultado.join(resultadoDec)
resultado = resultado.join(resultadoFec)

## Planilha Conjuntos Semelhantes DEC
semelhantesDEC = conjSemDec.join(data[['Conjunto','Distribuidora']+atrDec], on='CodConjSemeDEC')
semelhantesDEC = semelhantesDEC.rename(columns={'Conjunto': 'ConjSemeDEC', 'Distribuidora': 'DistSemeDEC'})

## Planilha Conjuntos Semelhantes FEC
semelhantesFEC = conjSemFec.join(data[['Conjunto','Distribuidora']+atrFec], on='CodConjSemeFEC')
semelhantesFEC = semelhantesFEC.rename(columns={'Conjunto': 'ConjSemeFEC', 'Distribuidora': 'DistSemeFEC'})

##### Exportação do resultado

In [15]:
# Planilha TRAJETORIA.xlsx
trajLimites.to_excel(caminho + 'TRAJETORIA'+ sufixo +'.xlsx', index=True)

# Planilha RESULTADO.xlsx
with pd.ExcelWriter(
    path = caminho + 'RESULTADO'+ sufixo +'.xlsx',
) as writer:
    resultado.to_excel(writer, sheet_name="Resultado", index=True, na_rep='NaN')
    columnLength=[9,37,9,18,25,20,14,37,18,12,25,20,14,37,18,12]
    for idx, val in enumerate(columnLength):
        writer.sheets['Resultado'].set_column(idx, idx, val)

# Planilha CONJUNTOSSEMELHANTES_DEC.xlsx
with pd.ExcelWriter(
    path = caminho + 'CONJUNTOSSEMELHANTES_DEC'+ sufixo +'.xlsx',
) as writer:
    semelhantesDEC.to_excel(writer, sheet_name="SemelhantesDEC", index=False, na_rep='NaN')
    columnLength=[17,37,20,17,37,18,10,9,12,12,13,9]
    for idx, val in enumerate(columnLength):
        writer.sheets['SemelhantesDEC'].set_column(idx, idx, val)

# Planilha CONJUNTOSSEMELHANTES_FEC.xlsx
with pd.ExcelWriter(
    path = caminho + 'CONJUNTOSSEMELHANTES_FEC'+ sufixo +'.xlsx',
) as writer:
    semelhantesFEC.to_excel(writer, sheet_name="SemelhantesFEC", index=False, na_rep='NaN')
    columnLength=[17,37,20,17,37,18,10,9,12,12,13,9]
    for idx, val in enumerate(columnLength):
        writer.sheets['SemelhantesFEC'].set_column(idx, idx, val)